In [1]:
import pandas as pd
import requests
from tqdm.auto import tqdm
from collections import defaultdict

In [49]:
headers = {'X-Requested-With': 'XMLHttpRequest'}
payload = ({'region':'77'})
url = 'http://www.rosneft-azs.ru/map_search'
r = requests.post(url, data=payload, headers=headers)

In [50]:
res = r.json()

In [51]:
stations = res['stations']

In [52]:
# stations[0]

In [53]:
url = 'http://www.rosneft-azs.ru/map_search'

In [54]:
# Формируем словарь со списками для колонок таблицы
# tab_dict = {'Azs':[],'Address' :[],'Price92':[],'Price95':[]}
tab_dict = defaultdict(list)
# Заполняем списки с помощью цикла
for station in tqdm(stations):
    tab_dict['Azs'].append(station['id'])
    tab_dict['Address'].append(station['address'])
    for d in station['price']:
        if d['type'] == 92:
            tab_dict['Price92'].append(d['price'])
        elif d['type'] == 95:
            tab_dict['Price95'].append(d['price'])

  0%|          | 0/2337 [00:00<?, ?it/s]

In [55]:
rn = pd.DataFrame.from_dict(tab_dict)
rn.head()

,Azs,Address,Price92,Price95
0,58520,"Ростовская обл., Мясниковский р-н, п. Чалтырь,...",47.04,0.00
1,58316,"Владимирская обл., Судогодский р-н, д. Лаврово...",45.70,49.20
2,58519,"Ростовская обл., г. Ростов-на-Дону, ул. 40 Лет...",47.04,51.44
3,58509,"Калужская обл., г. Калуга, ул. Зерновая, 34",44.40,47.50
4,58511,"Калужская обл., Боровский р-н, г. Балабаново, ...",44.40,47.65


In [56]:
azs = rn.Azs.unique().tolist()

In [66]:
nums = [x*100 for x in range(len(azs))]

In [67]:
rn['nums'] = rn.Azs.map(dict(zip(azs, nums)))

In [69]:
rn_new = rn[['Azs','Address']].copy()

In [71]:
rn_new.Address = rn_new.Address.apply(lambda x:x.split(',')[0])

In [74]:
rn_new = rn_new.rename(columns = {'Address':'Region'})

In [79]:
pd.merge(rn, rn_new, left_on='Azs', right_on='Azs', how='inner')

,Azs,Address,Price92,Price95,nums,Region
0,58520,"Ростовская обл., Мясниковский р-н, п. Чалтырь,...",47.04,0.00,0,Ростовская обл.
1,58316,"Владимирская обл., Судогодский р-н, д. Лаврово...",45.70,49.20,100,Владимирская обл.
2,58519,"Ростовская обл., г. Ростов-на-Дону, ул. 40 Лет...",47.04,51.44,200,Ростовская обл.
3,58509,"Калужская обл., г. Калуга, ул. Зерновая, 34",44.40,47.50,300,Калужская обл.
4,58511,"Калужская обл., Боровский р-н, г. Балабаново, ...",44.40,47.65,400,Калужская обл.
...,...,...,...,...,...,...
2332,100515,"Ульяновская обл., г. Ульяновск, пр-д 10-й Инже...",45.39,48.99,233200,Ульяновская обл.
2333,100517,"Ульяновская обл., г. Ульяновск, пр-т Нариманов...",45.39,48.99,233300,Ульяновская обл.
2334,105501,"Хабаровский край, Амурский район, п. Эльбан, у...",48.70,50.85,233400,Хабаровский край
2335,113433,"Ростовская область г.Ростов-на-Дону, Таганрогс...",46.04,50.44,233500,Ростовская область г.Ростов-на-Дону


In [19]:
','.join(rn.Address[0].split(',')[1:]).strip()

'Мясниковский р-н, п. Чалтырь, а/д М23, 9 км, слева'

In [15]:
rn['Region'] = rn.Address.apply(lambda x:x.split(',')[0])

In [20]:
rn['Address_2'] = rn.Address.apply(lambda x:','.join(x.split(',')[1:]).strip())

In [31]:
rn[
    (rn.Azs == 58520) | (rn.Azs == 58519)
]

,Azs,Address,Price92,Price95,Region,Address_2
0,58520,"Ростовская обл., Мясниковский р-н, п. Чалтырь,...",47.04,0.00,Ростовская обл.,"Мясниковский р-н, п. Чалтырь, а/д М23, 9 км, с..."
2,58519,"Ростовская обл., г. Ростов-на-Дону, ул. 40 Лет...",47.04,51.44,Ростовская обл.,"г. Ростов-на-Дону, ул. 40 Лет Победы, 2Г/51"


In [32]:
rn.loc[rn.Address.str.startswith('Рост'), ['Azs','Address']] = 0

In [33]:
rn.loc[rn.Address.str.startswith('Рост'), ['Azs','Address']]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [37]:
rn.iloc[2:4, 0]

2        0
3    58509
Name: Azs, dtype: int64

In [38]:
import os

In [45]:
os.mkdir('new_dir')

In [40]:
ls

fuels.ipynb  new_dir/     numpy.ipynb  sel.ipynb    soup.ipynb


In [41]:
os.remove('new_dir')

PermissionError: [Errno 1] Operation not permitted: 'new_dir'

In [12]:
# Добавить в таблицу другие виды топлива
#----- Ваш код здесь ----

In [ ]:
# Отделить регионы от адреса АЗС
#----- Ваш код здесь ----

In [ ]:
# Выбрать АЗС из Белоруссии и сохранить в rnbelarus
#----- Ваш код здесь ---

In [ ]:
# Пересчитать цены rnbelarus в рубли
#----- Ваш код здесь ----

In [ ]:
# Убрать АЗС из Белоруссии из таблциы rn
#----- Ваш код здесь ----

In [ ]:
# Расчитать средние цены на топливо по регионам
#----- Ваш код здесь ----

In [ ]:
# Создать директорию regional_prices
#----- Ваш код здесь ----

In [ ]:
# Записать в отдельный .csv файл цены для каждого региона.
# Разделить должен быть знак табуляции sep='\t'
# Название файла записать как название региона, заменив пробел на _
#----- Ваш код здесь ----

In [ ]:
# Выведите содержимое директории в переменную list_of_files
#----- Ваш код здесь ----

In [ ]:
# Создайте словарь regional_prices и заполните его таким образом, чтобы
# ключ был название файла без .csv, а значения - список строк, счиатанных из файла
#----- Ваш код здесь ----

In [ ]:
# Соберите таблицу rnnew из словаря regional_prices
#----- Ваш код здесь ----